In [4]:
from seeq import spy
import pandas as pd

In [3]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key')

# spy.search

Finds signals (tags), conditions (with capsules), scalars (constants), assets or any other type of item that Seeq indexes or keeps track of.

You will generally use this command before executing `spy.pull()`.

`spy.search(query, all_properties=False, workbook='Data Lab >> Data Lab Analysis')`

## Query Syntax

The `query` parameter is a dictionary of _property_: _filter_ values that are applied using AND logic by Seeq Server. Let's use the following examples to illustrate the important parts:

In [5]:
spy.search({
    'Name': 'Humid',
    'Path': 'Example >> Cooling Tower 1'
})

,Count,Time
Results,8,00:00:00.16


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name
0,D97692F2-A01D-4962-A75A-CEFB238124F9,Example >> Cooling Tower 1,Area A,Relative Humidity,NaN,StoredSignal,%,Example Data
1,D9AD12D6-8E4F-4F87-A122-F84273DB9AF8,Example >> Cooling Tower 1,Area J,Relative Humidity,NaN,StoredSignal,%,Example Data
2,E1312087-DECE-4758-9BDA-84D9A69147B3,Example >> Cooling Tower 1,Area G,Relative Humidity,NaN,StoredSignal,%,Example Data
3,D093453F-A0D7-4F0D-8CC9-C9D7FA8FCABD,Example >> Cooling Tower 1,Area I,Relative Humidity,NaN,StoredSignal,%,Example Data
4,F1EE66B8-F0BC-4846-9F25-64AEB6698767,Example >> Cooling Tower 1,Area K,Relative Humidity,NaN,StoredSignal,%,Example Data
5,A301C01A-01AC-4340-8602-B683270BB9F2,Example >> Cooling Tower 1,Area H,Relative Humidity,NaN,StoredSignal,%,Example Data
6,22D41E80-7CC6-4AE6-85EF-3424164326D6,Example >> Cooling Tower 1,Area B,Relative Humidity,NaN,StoredSignal,%,Example Data
7,8F12BD8E-D124-4440-8DF2-12E604D55301,Example >> Cooling Tower 1,Area C,Relative Humidity,NaN,StoredSignal,%,Example Data


This query returns anything with `Humid` in its `Name` property that also lives somewhere under `Example >> Cooling Tower 1` in an asset tree.

There are several capabilities and some constraints associated with querying:

1. The `Name` and `Description` properties are queried upon with the same wildcard and RegEx support as the Data tab in Seeq Workbench: https://seeq12.atlassian.net/wiki/spaces/KB/pages/146472969/Searching+for+Items

2. The `Path` property is a _virtual_ property that specifies the path through an asset tree from its root, with `>>` as delimiters for each level in the asset tree.

3. The returned `Asset` property is just the leaf asset node in the tree. It can only be used in tandem with the `Path` property.

4. For the `Type` property, you can specify just `Signal`, `Condition` or `Scalar` if you want to return both _stored_ and _calculated_ items.

5. You can specify `Datasource Name` _or_ you can specify both `Datasource ID` and `Datasource Class` to differentiate between datasources with the same name.

6. You can filter on `Data ID`, which is a unique identifier that is generally the same across instances of Seeq Server.

7. You can filter on `Archived` and `Cache Enabled` using either `True` or `False` Python boolean values.


## Retrieving all properties

`spy.search()` only returns a subset of item properties as can be seen in the output above. If you want to retrieve all properties, use `all_properties=True`. Note that this can be an expensive (slow) operation for queries that return many rows.

In [6]:
spy.search({
    'Name': 'Area A_*Humid*',
    'Datasource Name': 'Example Data'
}, all_properties=True)

,Count,Time
Results,1,00:00:00.08


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Cache Enabled,Data ID,Datasource Class,Datasource ID,Interpolation Method,Key Unit Of Measure,Maximum Interpolation,Source Maximum Interpolation,Source Value Unit Of Measure,Sync Token,Scoped To
0,88FE23DF-3C54-41B9-B699-90E85D99C30E,Area A_Relative Humidity,NaN,StoredSignal,%,Example Data,false,[Tag] Area A_Relative Humidity.sim.ts.csv,Time Series CSV Files,Example Data,Linear,ns,2min,2min,%,2019-12-14T00:09:22.281418Z,NaN


## Workbook scoping

The `workbook=<workbook_path>` parameter allows you to include items in your results that are scoped to a particular workbook. If you exclude the argument, then by default your search results will include both globally-scoped items and workbook-scoped items from the **Data Lab >> Data Lab Analysis** workbook (folder path delimiter by `>>`, workbook name at the end). This workbook generally will only exist if you have previous called `spy.push()`. This default behavior forces your `spy.push()` and subsequent `spy.search()` activities to be _sandboxed_, meaning that they will only be visible to you within a particular workbook.

If you want to only return globally-scoped items, specify `workbook=None` as the argument.

Another option is to specify `Scoped To` within your query block. You must supply a Workbook ID -- not a workbook path like the `workbook` parameter. This approach will limit your search to just those items scoped to a particular workbook (it will _exclude_ globally-scoped items).

## DataFrame as input

Instead of a Python dictionary for the `query` parameter, you can supply a Pandas DataFrame.

This is generally useful when you have a DataFrame full of tag names but don't know the Seeq `ID` value and therefore can't retrieve data via `spy.pull()`. Calling `spy.search(data_frame)` effectively "fills in" the `ID` field for you wherever possible.

The column headers specify the properties to search on and the column values specify the match criteria.

If you don't specify wildcards or use a RegEx, the match must be exact. (This behavior is in contrast to the dictionary case, where the non-wildcard/RegEx match is a "contains" comparison.) It is assumed that your DataFrame property values should match exactly so that you can have a large set of items to query for and there won't be ambiguity between item names like `F1843CC` and `F1843CC.SP`.

In [7]:
my_items = pd.DataFrame({
    'Name': [
        'Area A_Temperature',
        'Area B_Compressor Power',
        'Optimize'
    ],
    'Datasource Name': 'Example Data'
})

my_items

,Name,Datasource Name
0,Area A_Temperature,Example Data
1,Area B_Compressor Power,Example Data
2,Optimize,Example Data


In [8]:
spy.search(my_items)

,Count,Time
Results,2,00:00:00.14


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name
0,77B92A6B-2254-403F-907F-926B6E30464C,Area A_Temperature,NaN,StoredSignal,°F,Example Data
1,7E55C661-2F1F-4103-ADA5-FC35E1F32199,Area B_Compressor Power,NaN,StoredSignal,kW,Example Data


Notice that there are no results with `Optimize` in the name because it does not exactly match any items.